# RSA密码破译

`Import the necesssary module`

In [18]:
from sympy.ntheory.modular import crt
import gmpy2
from Crypto.Util.number import *
import owiener

`Load data`

In [4]:
def load_data(datanum):
    with open(file="./test/data%d" % (datanum), mode='r') as file:
        n, e, c = file.read(256), file.read(256), file.read(256)
        n, e, c = int(n, 16), int(e, 16), int(c, 16)
        return n, e, c


`Attack RSA`

In [5]:
# receive n,e,c as parameters, return m and d
# 低加密指数攻击
# 已经尝试过, 跑了90min一个没跑出来
# 主要原因在于原文m开头添加了一系列的1
def attack1(n, e, c):
    if e > 2:
        return -1
    i = 0
    while 1:
        tmp = gmpy2.iroot(c + i * n, e)
        if tmp[1]:
            return tmp[0]
        else:
            i += 1


# 低加密指数广播攻击
def attack2(n, e, c):
    pass
    #TODO
    return -1


# 维纳攻击
def attack3(n, e, c):
    if e < 10000000:
        return -1
    d=owiener.attack(e,n)
    if d is None:
        print("Failed")
    else:
        print("Hacked d={}".format(d))
    
import math


In [6]:
def attack_broadcast(e,lst_c,lst_n):
    M=1
    for n in lst_n:
        M=M*n
    lst_M=[]
    for n in lst_n:
        lst_M+=[M//n]
    m_e=M
    for i in range(len(lst_c)):
        m_e+=lst_c[i]*lst_M[i]*invert(lst_M[i],lst_n[i])
        print(i)
    m_e=m_e%M
    m=iroot(m_e,e)
    i=0
    while not m[1]:
        m_e+=M
        i+=1
        m=iroot(m_e,e)
    return m

In [26]:
lst_c,lst_n=[],[]
for i in range(21):
    n,e,c=load_data(i)
    print(i,e)

0 3
1 65537
2 65537
3 65537
4 5
5 5
6 3
7 5
8 65537
9 65537
10 65537
11 46786465362686334917265996843779843233606992585424976481745055335468678697948774988450305612127967926533923268260412557000125153569622340353246096040604284883505587337829322949633637609180797447754513992039018904786537115087888005528547900640339270052628915440787357271345416818313808448127098885767015748889
12 65537
13 5
14 152206992575706893484835984472544529509325440944131662631741403414037956695665533186650071476146389737020554215956181827422540843366433981607643940546405002217220286072880967331118344806315756304650248634546597784597963886656422706197757265316981889118026978865295597135470735576032282694348773714479076093197
15 65537
16 65537
17 3
18 5
19 65537
20 65537
